In [1]:
import subprocess
import time

import numpy as np
from matplotlib import pyplot as plt

Y1 = 550
Y2 = 1850
X1 = 1350
X2 = 2650
DEVERNAY_DOWNSCALE = 4

In [2]:
start_time = time.time()
w = X2 - X1
h = Y2 - Y1
total_frames = 300
step = 28
command = [
            "ffmpeg",
            "-y",
            "-i", "/Users/fathe/OneDrive/Documents/UK/MFF/Thesis/input/srp_GX010027.MP4",
            # "-vf",f"crop={w}:{h}:{X1}:{Y1},scale={w // DEVERNAY_DOWNSCALE}:{h // DEVERNAY_DOWNSCALE},format=gray,select='not(mod(n\\,{step}))'",
            # "-vsync", "vfr",  # very important for select to work
            "-vframes", str(total_frames),
            "-pix_fmt", "gray",
            "-f", "rawvideo",
            "pipe:1"
        ]
pipe = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.DEVNULL)

# frame_size = int((w // DEVERNAY_DOWNSCALE) * (h // DEVERNAY_DOWNSCALE))
frame_size = 3840 * 2160
frames = []

while True:
    raw = pipe.stdout.read(frame_size)
    if not raw or len(raw) < frame_size:
        break
    # frame = np.frombuffer(raw, dtype=np.uint8).reshape(
    #     (h // DEVERNAY_DOWNSCALE, w // DEVERNAY_DOWNSCALE)
    # )
    frame = np.frombuffer(raw, dtype=np.uint8).reshape((3840, 2160))
    frames.append(frame)

pipe.stdout.close()
pipe.wait()

print(time.time() - start_time)
frames = np.array(frames)

In [10]:
frames.shape

In [4]:
plt.figure(figsize=(10, 5))
plt.imshow(frames[0]-frames[4], cmap='gray')
plt.axis('off')
plt.show()

In [3]:
import psutil

ram_bytes = psutil.virtual_memory().total
ram_gb = ram_bytes / (1024 ** 3)

print(f"Total RAM: {ram_gb:.2f} GB")
